# Orientation/Direction Tuning
- Compute tuning curves.
- Determine orientation/direction selectivity of cells using permutation test (Ecker et al., 2014, Baden et al., 2016).

In [27]:
import numpy as np
import pandas as pd

import datajoint as dj

from lib.data import *
from lib.tuning import *
from lib.calcium import *
from lib.plot import *
from lib.utils import *

## Load data tables

In [28]:
Stimulus = load_dict("data/Stimulus.pkl")
EASETrace = load_dict("data/EASETrace.pkl")
EASETuning = load_dict("data/EASETuning.pkl")

## Pyramidal cells with functional data

In [29]:
seg_id_list = EASETuning["segment_id"]
scan_list = EASETuning["scan_id"]
n_cells = seg_id_list.shape[0]

## Determine orientation/direction tuned cells with permutation test
- $p<0.01$ are significantly tuned cells.

In [30]:
T = 10000 # Number of iterations for permutation test

dsi_list = np.zeros(n_cells)
osi_list = np.zeros(n_cells)
dsi_pvalue = np.zeros(n_cells)
osi_pvalue = np.zeros(n_cells)
for i in range(n_cells):
    
    seg_id = seg_id_list[i]
    scan_id = int(scan_list[seg_id_list==seg_id])

    trace = get_trace(EASETrace, seg_id, scan_id, "trace_raw")
    stimlab = get_stim_label(Stimulus, scan_id)
    
    response_array = get_peakamp_tdarray(trace, stimlab)
    tune = tuning_curve(response_array)
    
    dsi_list[i] = dsi(tune)
    osi_list[i] = osi(tune)
    dsi_shuf, dsi_pvalue[i] = permutation_test(response_array, T, "dsi")
    osi_shuf, osi_pvalue[i] = permutation_test(response_array, T, "osi")
    
    if (i+1)%10==0:
        print("{} / {} complete.".format(i+1, n_cells))

10 / 112 complete.
20 / 112 complete.
30 / 112 complete.
40 / 112 complete.
50 / 112 complete.
60 / 112 complete.
70 / 112 complete.
80 / 112 complete.
90 / 112 complete.
100 / 112 complete.
110 / 112 complete.


In [5]:
# Same format as data/cell_functional_info.csv
data = {"segment_id": seg_id_list,
     "scan": scan_list,
     "osi": osi_list, 
     "osi_p": osi_pvalue,
     "dsi": dsi_list,
     "dsi_p": dsi_pvalue}

df = pd.DataFrame(data=data)
df

,segment_id,scan,osi,osi_p,dsi,dsi_p
0,648518346349539895,1,0.857341,0.079,0.220618,0.918
1,648518346349537860,1,0.851261,0.000,0.180522,0.476
2,648518346349538440,1,0.893301,0.223,0.167758,0.994
3,648518346349538527,1,0.982902,0.504,0.935960,0.941
4,648518346349538209,1,0.901787,0.000,0.048472,0.924
...,...,...,...,...,...,...
107,648518346349537741,5,0.906518,0.060,0.378011,0.957
108,648518346349537901,5,0.933251,0.162,0.932056,0.187
109,648518346349538001,5,0.920036,0.127,0.921915,0.101
110,648518346349538251,5,0.801713,0.141,0.780844,0.172
